<a href="https://colab.research.google.com/github/chdasiegel/Dynasty/blob/main/student_mle_miniproject_recommendation_engines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [ ]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).  
#(This colab was depricated)

In [ ]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [ ]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?

In [ ]:
#inspect ratings and movies DataFrame
ratings.head(), ratings.info()
movies.head(), movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   user_id         100000 non-null  int64
 1   movie_id        100000 non-null  int64
 2   rating          100000 non-null  int64
 3   unix_timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   title               1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   imdb_url            1679 non-null   object 
 5   genre_unknown       1682 non-null   int64  
 6   Action              1682 non-null   int64  
 7   Adventure           1682 non-null   

(   movie_id              title release_date  video_release_date  \
 0         1   Toy Story (1995)  01-Jan-1995                 NaN   
 1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
 2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
 3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
 4         5     Copycat (1995)  01-Jan-1995                 NaN   
 
                                             imdb_url  genre_unknown  Action  \
 0  http://us.imdb.com/M/title-exact?Toy%20Story%2...              0       0   
 1  http://us.imdb.com/M/title-exact?GoldenEye%20(...              0       1   
 2  http://us.imdb.com/M/title-exact?Four%20Rooms%...              0       0   
 3  http://us.imdb.com/M/title-exact?Get%20Shorty%...              0       1   
 4  http://us.imdb.com/M/title-exact?Copycat%20(1995)              0       0   
 
    Adventure  Animation  Children  ...  Fantasy  Film-Noir  Horror  Musical  \
 0          0          1    

In [ ]:
#Determine number of unique values for users and movies
print("Unique users:", ratings['user_id'].nunique())
print("Unique movies:", ratings['movie_id'].nunique())

Unique users: 943
Unique movies: 1682


In [ ]:
# Merge movies and ratings dataframes

In [ ]:
ratings_merged = ratings.merge(movies, on='movie_id', how='left')

print("Merged dataframe shape:", ratings_merged.shape)
ratings_merged.head()


Merged dataframe shape: (100000, 27)


,user_id,movie_id,rating,unix_timestamp,title,release_date,video_release_date,imdb_url,genre_unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%...,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-11...,0,0,...,0,0,0,0,0,0,0,0,0,0


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [ ]:
# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df):
  # Get the user's rated movies

  user_ratings = df[df['user_id'] == user_id]
  user_movie_ids = user_ratings['movie_id'].unique()

  if len(user_movie_ids) == 0:
    return pd.DataFrame(columns=['movie_id', 'title'])

  # Create a TF-IDF matrix using movie genres

  genre_matrix = movies[genre_cols].values

  # Compute the cosine similarity between movie genres

  cosine_sim = cosine_similarity(genre_matrix)

  # Get the indices of the similar movies based on cosine similarity

  sim_scores = np.zeros(len(movies))

  movie_index = pd.Series(movies.index, index=movies['movie_id'])

  for m_id in user_movie_ids:
    if m_id in movie_index:
        idx = movie_index[m_id]
        sim_scores += cosine_sim[idx]

  # Remove duplicates and movies already rated by the user

  rated_indices = movie_index[user_movie_ids]
  sim_scores[rated_indices] = 0  # already watched films


  # Return recommendations

  top_indices = np.argsort(sim_scores)[::-1][:5]

  recs = movies.loc[top_indices, ['movie_id', 'title']].copy()
  recs['similarity_score'] = sim_scores[top_indices]

  return recs





In [ ]:
#Test

test_user = ratings_merged['user_id'].iloc[379]
content_based_recommendation(test_user, ratings_merged)


,movie_id,title,similarity_score
73,74,Faster Pussycat! Kill! Kill! (1965),103.261808
336,337,"House of Yes, The (1997)",101.891604
1102,1103,Trust (1990),101.163053
1630,1631,"Slingshot, The (1993)",101.163053
1185,1186,"Inkwell, The (1994)",101.163053


In [ ]:
#Analyze

recs = content_based_recommendation(test_user, ratings_merged)

for _, row in recs.iterrows():
    movie_id = row['movie_id']
    movie_row = movies[movies['movie_id'] == movie_id]

    # Extract genre names
    genres = [g for g in genre_cols if movie_row[g].values[0] == 1]

    print(f"{row['title']}: {genres}")


Faster Pussycat! Kill! Kill! (1965): ['Action', 'Comedy', 'Drama']
House of Yes, The (1997): ['Comedy', 'Drama', 'Thriller']
Trust (1990): ['Comedy', 'Drama']
Slingshot, The (1993): ['Comedy', 'Drama']
Inkwell, The (1994): ['Comedy', 'Drama']


Above results conclude that user 379 prefers **Drama** and/or **Comedy** first, followed by **Action** and then **Thriller**

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [ ]:
# Collaborative Filtering using User-Item Interactions

def collaborative_filtering_recommendation(user_id, df):

  # Create the user-item matrix

  user_item = df.pivot_table(index='user_id', columns='movie_id', values='rating')

  # Fill missing values with 0 (indicating no rating)

  user_item_filled = user_item.fillna(0)

  # Calculate user-user similarity matrix using cosine similarity

  sim_matrix = cosine_similarity(user_item_filled)
  sim_df = pd.DataFrame(sim_matrix, index=user_item_filled.index, columns=user_item_filled.index)

  # Get the similarity scores of the target user with all other users

  if user_id not in sim_df.index:
    return pd.DataFrame(columns=['movie_id', 'title'])

  sim_scores = sim_df.loc[user_id].drop(user_id)

  # Find the top N most similar users (excluding the target user)

  top_users = sim_scores.sort_values(ascending=False).head(5).index

  # Generate movie recommendations based on the most similar users

  neighbor_ratings = user_item_filled.loc[top_users]
  weights = sim_scores.loc[top_users].values

  weighted_sum = neighbor_ratings.T.dot(weights)
  weight_totals = (neighbor_ratings.notna().T * weights).sum(axis=1)

  with np.errstate(divide='ignore', invalid='ignore'):
    scores = weighted_sum / weight_totals

  scores = scores.fillna(0)

  # Remove duplicates from recommendations

  top_movie_ids = scores.sort_values(ascending=False).head(5).index

  # Build recommendations dataframe

  recs = movies[movies['movie_id'].isin(top_movie_ids)][['movie_id', 'title']].copy()
  recs = recs.set_index('movie_id').loc[top_movie_ids].reset_index()  # keep order
  recs['score'] = scores.loc[top_movie_ids].values

  return recs


In [ ]:
#Test

test_user = ratings_merged['user_id'].iloc[379]
collaborative_filtering_recommendation(test_user, ratings_merged)


,movie_id,title,score
0,64,"Shawshank Redemption, The (1994)",5.000000
1,318,Schindler's List (1993),5.000000
2,127,"Godfather, The (1972)",5.000000
3,181,Return of the Jedi (1983),4.606427
4,50,Star Wars (1977),4.606427


In [ ]:
#Analyze

recs = collaborative_filtering_recommendation(test_user, ratings_merged)

for _, row in recs.iterrows():
    movie_id = row['movie_id']
    movie_row = movies[movies['movie_id'] == movie_id]

    # Extract genre names
    genres = [g for g in genre_cols if movie_row[g].values[0] == 1]

    print(f"{row['title']}: {genres}")


Shawshank Redemption, The (1994): ['Drama']
Schindler's List (1993): ['Drama', 'War']
Godfather, The (1972): ['Action', 'Crime', 'Drama']
Return of the Jedi (1983): ['Action', 'Adventure', 'Romance', 'Sci-Fi', 'War']
Star Wars (1977): ['Action', 'Adventure', 'Romance', 'Sci-Fi', 'War']


Above results conclude that user 379 prefers **Drama** followed by **Adventure**. **War, Crime, Romance, Sci-Fi** and **Action** are non factors

Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

In [ ]:
# Test the recommendation engines

In [ ]:
# Pick 3 unique user_ids at random
user_ids = ratings_merged['user_id'].drop_duplicates().sample(3, random_state=42)

for uid in user_ids:
    print("=" * 60)
    print(f"USER ID: {uid}")
    print("- Content-based recommendations -")
    display(content_based_recommendation(uid, ratings_merged))

    print("\n- Collaborative filtering recommendations -")
    display(collaborative_filtering_recommendation(uid, ratings_merged))
    print("\n")


USER ID: 262
- Content-based recommendations -


,movie_id,title,similarity_score
511,512,Wings of Desire (1987),68.872445
935,936,Brassed Off (1996),68.872445
1099,1100,What Happened Was... (1994),68.872445
169,170,Cinema Paradiso (1988),68.872445
691,692,"American President, The (1995)",68.872445



- Collaborative filtering recommendations -


,movie_id,title,score
0,98,"Silence of the Lambs, The (1991)",5.000000
1,318,Schindler's List (1993),5.000000
2,97,Dances with Wolves (1990),4.810218
3,12,"Usual Suspects, The (1995)",4.791980
4,191,Amadeus (1984),4.602198




USER ID: 136
- Content-based recommendations -


...

,movie_id,title,score
0,14,"Postino, Il (1994)",4.584697
1,100,Fargo (1996),4.437015
2,286,"English Patient, The (1996)",4.390066
3,124,Lone Star (1996),4.390066
4,275,Sense and Sensibility (1995),3.989579




USER ID: 821
- Content-based recommendations -


,movie_id,title,similarity_score
3,4,Get Shorty (1995),24.280357
73,74,Faster Pussycat! Kill! Kill! (1965),24.280357
774,775,Something to Talk About (1995),24.149053
516,517,Manhattan (1979),24.149053
777,778,Don Juan DeMarco (1995),24.149053



- Collaborative filtering recommendations -


,movie_id,title,score
0,174,Raiders of the Lost Ark (1981),5.000000
1,318,Schindler's List (1993),4.800984
2,69,Forrest Gump (1994),4.603254
3,71,"Lion King, The (1994)",4.407361
4,121,Independence Day (ID4) (1996),4.405649


## Overall Comparison: Content-Based vs Collaborative Filtering

Across the three users, the two methods show clear differences in how they generate recommendations:

### Content-Based Filtering (Genres)

- Focuses on **movie genres** and recommends films with similar attributes to what the user has rated before.
- It may recommend less mainstream titles if they match the user’s genre profile.
- Useful for keeping recommendations aligned with the user’s established preferences.

### Collaborative Filtering (User–User Similarity)

- Looks for **users with similar rating patterns** and recommends movies that those users rated highly.
- Frequently highlights **popular, highly rated, or well-known films**.
- Not limited to genre, so recommendations can cover a wider range of movie types.
- Helps recommend movies the user hasn’t seen but are favored by similar viewers.

### Summary

- Content-based filtering is driven by **movie characteristics** and stays close to the user’s past preferences.
- Collaborative filtering is driven by **similar user behavior** and tends to recommend well-liked or broadly appreciated films.
- Combining both approaches would create a balanced recommendation system:
  - Content-based maintains consistency with user taste.
  - Collaborative filtering broadens the selection using similar-user insights.
